In [7]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

In [8]:
train_trans = transforms.Compose([
    # 1. Geometric Transformations (with resizing)
    transforms.Resize((256, 256)),
    transforms.RandomHorizontalFlip(p=0.5),  # Horizontal flips for left-right consistency
    transforms.RandomRotation(10),  # Minor rotation for realistic variation
    transforms.RandomAffine(
        degrees=0,  # No additional rotation
        translate=(0.05, 0.05),  # Small positional variance
        scale=(0.9, 1.1)  # Conservative scaling for proportion preservation
    ),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.3, interpolation=3),  # Subtle 3D perspective

    # 2. Color Augmentations
    transforms.ColorJitter(
        brightness=0.15, contrast=0.15, saturation=0.15, hue=0.05
    ),  # Mild color variation for natural lighting

    # 3. Conversion and Normalization
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5597623586654663, 0.5584744215011597, 0.5615870952606201],
        std=[0.2830338776111603, 0.27588963508605957, 0.2759666442871094]
    )
])

In [9]:
train_dataset = ImageFolder(root='/Users/arponbiswas/Computer-Vision-Projects/Image_classification_projects/PC_Parts_Image_Classification/Data/pc_parts_ready', transform=train_trans)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [10]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.version = '1.0'
        self.layer1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=6, kernel_size=6, stride=4, padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3),
        )
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(6 * 21 * 21, 14)

    def forward(self, x):
        out = self.layer1(x)
        out = self.flatten(out)
        out = self.fc1(out)
        return out

In [11]:
model = SimpleCNN()

In [12]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [14]:
def training_analysis(model, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0.0
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(images)
            loss = loss_fn(outputs, labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss/len(train_loader):.4f}")

In [ ]:
training_analysis(model, epochs=1)

Epoch [1/3], Loss: 2.2338
Epoch [2/3], Loss: 2.1605
Epoch [3/3], Loss: 2.1403
